In [1]:
import re
from zhon.hanzi import punctuation
import numpy as np
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
import editdistance
import pickle
from tqdm import tqdm

In [2]:
with open('/Users/mozhiwen/Documents/Data_source/chinese_stopwords.txt','r') as file:
    stopwords=[i[:-1] for i in file.readlines()]

In [3]:
news = pd.read_csv('sqlResult_1558435.csv',encoding='gb18030')

In [4]:
news.shape

(89611, 7)

In [5]:
news.head(5)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [6]:
#nas in the dataset
news[news.content.isna()].head(5)

,id,author,source,content,feature,title,url
100,89517,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",天和防务股东未来6个月内计划减持不超过480万股公司股份,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
103,89514,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",晶盛机电调整限制性股票回购价格,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
997,88620,NaN,央广网,NaN,"{""type"":""时事要闻"",""site"":""参考消息"",""commentNum"":""0"",...",[主播不在家]第二季：主播陈亮体验垃圾清运,http://www.cankaoxiaoxi.com/china/20170623/214...
1273,88344,NaN,央广网,NaN,"{""type"":""IT业界"",""site"":""参考消息"",""commentNum"":""0"",...",LKK洛可可：想象力经济时代或已到来,http://www.cankaoxiaoxi.com/science/20170610/2...
1282,88335,NaN,央广网,NaN,"{""type"":""IT业界"",""site"":""参考消息"",""commentNum"":""0"",...",CES2017：京东发布两款叮咚智能音箱新品 开放Alpha平台,http://www.cankaoxiaoxi.com/science/20170610/2...


In [7]:
#drop the nas
news=news.dropna(subset=['content'])

In [8]:
news.shape

(87054, 7)

In [9]:
news[news.source=='新华社'].shape

(78661, 7)

In [10]:
78661/87054

0.9035885772049532

In [11]:
def split_text(text):return ' '.join([w for w in list(jieba.cut(re.sub('\s|[%s]' % (punctuation),'',text))) if w not in stopwords])

In [12]:
split_text(news.iloc[1].content)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/vr/sknbl6g920vgq7zp6cb5w76c0000gn/T/jieba.cache
Loading model cost 0.912 seconds.
Prefix dict has been built succesfully.


'骁龙 835 唯一 Windows10 桌面 平台 认证 ARM 处理器 高通 强调 不会 只 考虑 性能 屏蔽掉 小 核心 相反 正 联手 微软 找到 一种 适合 桌面 平台 兼顾 性能 功耗 完美 方案 报道 微软 已经 拿到 一些 源码 Windows10 更好 理解 big little 架构 资料 显示 骁龙 835 一款 集成 CPUGPU 基带 蓝牙 Wi Fi SoC 传统 Wintel 方案 节省 至少 30% PCB 空间 按计划 今年 Q4 华硕 惠普 联想 首发 骁龙 835Win10 电脑 预计 均 二合一 形态 产品 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台'

In [13]:
jieba.enable_parallel(4)

In [14]:
#prepare data for machine learning
#corpus=list(map(split_text,[str(i) for i in news.content]))
with open('corpus.pkl','rb') as file:
    corpus = pickle.load(file)

In [15]:
jieba.disable_parallel()

with open('corpus.pkl','wb') as file:
    pickle.dump(corpus,file)

In [16]:
countvectorizer = CountVectorizer(encoding='gb18030',min_df=0.015)
tfidftransformer = TfidfTransformer()
clf = MultinomialNB()

In [17]:
countvector = countvectorizer.fit_transform(corpus)

In [18]:
countvector.shape

(87054, 883)

In [19]:
tfidf = tfidftransformer.fit_transform(countvector)

In [20]:
tfidf.shape

(87054, 883)

In [135]:
label=list(map(lambda source: 1 if source == '新华社' or source == '新华网' else 0,news.source))

In [136]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(tfidf,label,test_size = 0.3, random_state=42)

In [137]:
clf.fit(X=X_train.toarray(),y=y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [138]:
scores=cross_validate(clf,X_train.toarray(),y_train,scoring=('precision','recall','accuracy','f1'),cv=3,return_train_score=True)

In [139]:
scores

{'fit_time': array([0.42017913, 0.47715306, 0.45012403]),
 'score_time': array([0.16519976, 0.1802392 , 0.18391705]),
 'test_precision': array([0.95986699, 0.96215288, 0.96188664]),
 'train_precision': array([0.96223764, 0.96166327, 0.96263591]),
 'test_recall': array([0.91058414, 0.91395627, 0.90737014]),
 'train_recall': array([0.91264855, 0.91071769, 0.91012183]),
 'test_accuracy': array([0.88460592, 0.8895781 , 0.88360987]),
 'train_accuracy': array([0.88851418, 0.88632336, 0.88667356]),
 'test_f1': array([0.93457631, 0.9374355 , 0.93383341]),
 'train_f1': array([0.9367873 , 0.93549739, 0.9356426 ])}

In [140]:
y_predict = clf.predict(X_test.toarray())

In [141]:
def show_test_reslt(y_true,y_pred):
    print('accuracy:',accuracy_score(y_true,y_pred))
    print('precison:',precision_score(y_true,y_pred))
    print('recall:',recall_score(y_true,y_pred))
    print('f1_score:',f1_score(y_true,y_pred))

In [142]:
show_test_reslt(y_test,y_predict)

accuracy: 0.8900333116360991
precison: 0.9625166740773677
recall: 0.9143014022638959
f1_score: 0.9377897153749513


## Use the model to detect copy news

------------------

In [143]:
prediction = clf.predict(tfidf.toarray())

labels = np.array(label)

In [144]:
compare_news_index = pd.DataFrame({'prediction':prediction,'labels':labels})

In [145]:
copy_news_index=compare_news_index[(compare_news_index['prediction'] == 1) & (compare_news_index['labels'] == 0)].index

In [245]:
xinhuashe_news_index=compare_news_index[(compare_news_index['labels'] == 1)].index

In [146]:
len(copy_news_index)

2813

- In sklearn, in order to use cosine similarity in kmeans, we should scale the data to unit norm then use the kmeans class

In [195]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from collections import defaultdict

In [196]:
normalizer = Normalizer()

In [197]:
scaled_array = normalizer.fit_transform(tfidf.toarray())

In [342]:
kmeans = KMeans(n_clusters=25,random_state=42,n_jobs=-1)
k_labels = kmeans.fit_predict(scaled_array)

In [344]:
id_class = {index:class_ for index,class_ in enumerate(k_labels)}

In [345]:
class_id = defaultdict(set)
for index,class_ in id_class.items():
    if index in xinhuashe_news_index.tolist():
        class_id[class_].add(index)

In [346]:
count=0
for k in class_id:
    print(len(class_id[k]),count)
    count +=1

11755 0
10067 1
5966 2
3527 3
3986 4
1753 5
1497 6
2499 7
1551 8
722 9
2257 10
1743 11
1565 12
3032 13
5840 14
4066 15
3617 16
875 17
1696 18
1591 19
766 20
2900 21
2770 22
1238 23
1554 24


In [370]:
def print_element(ids,n=10):
    count=0
    for i in ids:
        if count > 10: break
        count+=1
        print(news.iloc[i].content,'\n')

In [371]:
for k in class_id:
    print(k,'\n')
    print_element(class_id[k],5)
    

2 

这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄
 

新华社照片，邯郸（河北），2017年5月19日\n合理营养健康成长\n5月19日，河北省邯郸市邯山区美的小学学生制作蔬菜、水果拼盘。\n当日，河北省邯郸市邯山区美的小学开展“我是小小营养师”活动，学生们动手制作蔬菜、水果拼盘，在游戏中了解各种蔬菜、水果的营养价值，学习科学的饮食知识。\n新华社记者牟宇摄 

新华社照片，开罗，2017年4月20日\n（国际）（2）第五届埃及国际鼓乐节在开罗举行\n4月20日，在埃及首都开罗，艺术家们在埃及国际鼓乐节上表演。\n当日，第五届埃及国际鼓乐节在埃及首都开罗开幕，来自23个国家和地区的艺术家们齐聚开罗萨拉丁城堡，为现场的近千名观众献上了精彩的表演。\n新华社记者赵丁喆摄 

新华社照片，开罗，2017年4月20日\n（国际）（1）第五届埃及国际鼓乐节在开罗举行\n4月20日，在埃及首都开罗，艺术家们在埃及国际鼓乐节上表演。\n当日，第五届埃及国际鼓乐节在埃及首都开罗开幕，来自23个国家和地区的艺术家们齐聚开罗萨拉丁城堡，为现场的近千名观众献上了精彩的表演。\n新华社记者赵丁喆摄 

\n新华社5月12日至5月18日国内新闻照片一周精选。\n图片编辑|李梦娇;覃海石\n\n“我国海域天然气水合物试采成功”\n“蓝鲸一号”钻探平台上喷出的火焰（5月16日摄）。由国土资源部中国地质调查局组织实施的我国海域天然气水合物试采在南海神狐海域实现连续8天稳定产气，试采取得圆满成功，实现了我国天然气水合物开发的历史性突破。;新华社/梁旭\n\n“和妈妈在一起”\n5月13日，在山东省临沂市郯城县，一名年轻妈妈亲吻自己的儿子，夕阳下的母子剪影折射出暖暖的亲情。;新华社（张春雷）\n\n;;;;;;;“折翼的天使”\n;;;;;;;5月17日，蒋张子怡（上）和母亲张水香一起在学校的舞蹈房内练习舞蹈。今年12岁的蒋张子怡是江西上饶市信州区第十小学六年级的一名学生。2010年，车祸夺去了她的双腿，但却没有磨灭她对舞蹈的信念和渴望。多年来，对于舞蹈，蒋张子怡始终乐此不疲，希望将来可以成为一名舞蹈家。她告诉记者，舞蹈可以拉近自己和别人的距离，获得他人的尊重。新华社/胡晨欢;;;;;;;“实弹发射”;;;;;;;火箭军在戈壁训练场组织实战化训练，进行实弹

In [347]:
def find_similar_text(cpindex,top=10):
    dist_dict={i:editdistance.eval(corpus[cpindex],corpus[i]) for i in class_id[id_class[cpindex]]}
    return sorted(dist_dict.items(),key=lambda x:x[1])[:top]

In [348]:
copy_news_index.tolist()

[4,
 24,
 28,
 30,
 32,
 35,
 37,
 43,
 45,
 49,
 50,
 57,
 59,
 63,
 66,
 69,
 71,
 75,
 79,
 81,
 87,
 88,
 93,
 99,
 103,
 105,
 107,
 109,
 114,
 115,
 120,
 121,
 122,
 123,
 126,
 127,
 130,
 131,
 133,
 134,
 135,
 139,
 140,
 141,
 142,
 144,
 145,
 146,
 147,
 150,
 152,
 153,
 154,
 156,
 159,
 160,
 161,
 162,
 163,
 166,
 168,
 169,
 171,
 173,
 174,
 178,
 184,
 187,
 192,
 195,
 199,
 201,
 204,
 205,
 207,
 208,
 212,
 213,
 214,
 215,
 217,
 221,
 224,
 227,
 228,
 231,
 233,
 235,
 237,
 239,
 247,
 248,
 249,
 258,
 259,
 261,
 273,
 277,
 285,
 289,
 290,
 293,
 300,
 305,
 309,
 313,
 314,
 315,
 316,
 318,
 319,
 331,
 332,
 335,
 336,
 339,
 340,
 342,
 345,
 348,
 350,
 351,
 353,
 354,
 355,
 356,
 358,
 361,
 368,
 369,
 373,
 377,
 381,
 384,
 385,
 388,
 389,
 390,
 391,
 392,
 394,
 395,
 397,
 399,
 402,
 404,
 405,
 406,
 409,
 415,
 417,
 421,
 422,
 424,
 428,
 430,
 431,
 433,
 435,
 437,
 438,
 439,
 441,
 442,
 447,
 449,
 452,
 455,
 456,
 457,
 458,

In [357]:
fst=find_similar_text(3352)

In [358]:
print('怀疑抄袭:\n')

print(news.iloc[3352].content)

print('相似原文:\n')

print(news.iloc[3134].content)

print('editdistince:',editdistance.eval(corpus[3352],corpus[3134]))

怀疑抄袭:

　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。
　　三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加
　　5月份，70个城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个。二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个。

相似原文:

　　国家统计局19日发布数据，5月份，15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落，其中9个城市环比下降或持平。这9个价格环比下降或持平的城市为：北京、上海、南京、杭州、合肥、福州、郑州、深圳、成都。
　　“5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。”国家统计局城市司高级统计师刘建伟说，从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建

In [360]:
#for i in fst:print(news.iloc[i[0]].content,i) 

In [289]:
from random import sample

In [291]:
dis_sort_dict={}
for i in tqdm(sample(copy_news_index.tolist(),300)):
    dis_sort_dict[i] = find_similar_text(i)


100%|██████████| 300/300 [19:24<00:00,  3.88s/it]
